In [1]:
import math
import random

#### Neural Network Specification

| Layer    | Nuerons          | Activation |
| -------- | ---------------- | ---------- |
| Input    | x1, x2, x3       | None       | 
| Hidden   | h1, h2, h3, h4, h5, h6, h7 | Sigmoid |
| Output   | y1 | Sigmoid |

We have a **3 layer neural network** with a **single hidden layer** containing 7 neurons.

In [2]:
# sigmoid function and its derivative
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

In [3]:
# initialize the weights randomly
def init_weights(num_inputs, num_hidden, num_outputs):
    W1 = [[random.uniform(-1, 1) for _ in range(num_inputs)] for _ in range(num_hidden)]
    W2 = [random.uniform(-1, 1) for _ in range(num_hidden)]
    return W1, W2

In [4]:
# feedforward step
def feedforward(X, W1, W2):

    # hidden layer
    hidden_input = [sum(X[i] * W1[j][i] for i in range(len(X))) for j in range(len(W1))]
    hidden_output = [sigmoid(h) for h in hidden_input]

    # output layer
    final_input = sum(hidden_output[i] * W2[i] for i in range(len(hidden_output)))
    final_output = sigmoid(final_input)

    return hidden_output, final_output

In [5]:
# backpropagation step
def backpropagation(X, Y, W1, W2, hidden_output, final_output, learning_rate):

    # calculate the error
    output_error = Y - final_output
    output_delta = output_error * sigmoid_derivative(final_output)

    # calculate the hidden layer error
    hidden_error = [output_delta * W2[i] for i in range(len(W2))]
    hidden_delta = [hidden_error[i] * sigmoid_derivative(hidden_output[i]) for i in range(len(hidden_error))]

    # update the weights for the output layer
    for i in range(len(W2)):
        W2[i] += hidden_output[i] * output_delta * learning_rate
    
    # update the weights for the hidden layer
    for i in range(len(W1)):
        for j in range(len(X)):
            W1[i][j] += X[j] * hidden_delta[i] * learning_rate

In [6]:
# train the neural network
def train_network(train_data, W1, W2, epochs = 10000, learning_rate = 0.1):
    for _ in range(epochs):
        for x, y in train_data:
            hidden_output, final_output = feedforward(x, W1, W2)
            backpropagation(x, y, W1, W2, hidden_output, final_output, learning_rate)
        
    return W1, W2

In [7]:
# Function to calculate accuracy of the neural network
def test_accuracy(train_data, W1, W2):
    correct_predictions = 0
    total_samples = len(train_data)

    for x, y in train_data:
        _, final_output = feedforward(x, W1, W2)
        predicted = round(final_output)  # Round the final output to 0 or 1
        if predicted == y:
            correct_predictions += 1

    accuracy = (correct_predictions / total_samples) * 100
    return accuracy


In [15]:
# verify the learned function
def verify_network(W1, W2, train_data, diagnostics = False):
    correct_predictions = 0
    total_samples = len(train_data)

    for x, y in train_data:
        _, final_output = feedforward(x, W1, W2)
        predicted = round(final_output)
        if predicted == y:
            correct_predictions += 1
        if diagnostics:
            print(f"Input: {x} => Predicted: {round(final_output)} (Expected: {int(y)})")
    
    accuracy = (correct_predictions / total_samples) * 100
    print(f"Accuracy: {accuracy:.2f}%")

#### Generating boolean functions and input data

In [9]:
# Define 5 different Boolean functions (other functions can be added)
boolean_functions = [
        lambda x1, x2, x3: x1 and x2 and x3,                          # AND
        lambda x1, x2, x3: x1 or x2 or x3,                            # OR
        lambda x1, x2, x3: x1 ^ x2 ^ x3,                              # XOR
        lambda x1, x2, x3: (x1 and x2) or x3 and (x2 or x1),          # Complex AND-OR
        lambda x1, x2, x3: (x1 or x2) and (not x3)                    # OR with NOT
    ]
    
# Generate input training data (all 8 combinations of 3 binary variables)
input_data = [
        [0, 0, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 1, 1],
        [1, 0, 0],
        [1, 0, 1],
        [1, 1, 0],
        [1, 1, 1],
]

#### Now we train and verify the network for each boolean function

In [16]:
# Train and test for each Boolean function
for idx, boolean_func in enumerate(boolean_functions):
   print(f"\nTraining for Boolean function {idx + 1}...")

   # Generate training data (input-output pairs)
   train_data = [(x, boolean_func(*x)) for x in input_data]
        
   # Initialize weights
   W1, W2 = init_weights(3, 7, 1)  # 3 inputs, 7 hidden neurons, 1 output neuron
        
   # Train the neural network
   W1, W2 = train_network(train_data, W1, W2)
        
   # Verify the network
   print(f"Verification for Boolean function {idx + 1}:")
   verify_network(W1, W2, train_data, diagnostics=True)


Training for Boolean function 1...
Verification for Boolean function 1:
Input: [0, 0, 0] => Predicted: 0 (Expected: 0)
Input: [0, 0, 1] => Predicted: 0 (Expected: 0)
Input: [0, 1, 0] => Predicted: 0 (Expected: 0)
Input: [0, 1, 1] => Predicted: 0 (Expected: 0)
Input: [1, 0, 0] => Predicted: 0 (Expected: 0)
Input: [1, 0, 1] => Predicted: 0 (Expected: 0)
Input: [1, 1, 0] => Predicted: 0 (Expected: 0)
Input: [1, 1, 1] => Predicted: 1 (Expected: 1)
Accuracy: 100.00%

Training for Boolean function 2...
Verification for Boolean function 2:
Input: [0, 0, 0] => Predicted: 0 (Expected: 0)
Input: [0, 0, 1] => Predicted: 1 (Expected: 1)
Input: [0, 1, 0] => Predicted: 1 (Expected: 1)
Input: [0, 1, 1] => Predicted: 1 (Expected: 1)
Input: [1, 0, 0] => Predicted: 1 (Expected: 1)
Input: [1, 0, 1] => Predicted: 1 (Expected: 1)
Input: [1, 1, 0] => Predicted: 1 (Expected: 1)
Input: [1, 1, 1] => Predicted: 1 (Expected: 1)
Accuracy: 100.00%

Training for Boolean function 3...
Verification for Boolean funct

We see that the network has successfully learned each of the functions.